# Engineered Data Quality Report

In this notebook we are going to create a data quality report piece-by-piece and then put it all together at the end.

In [5]:
# Import the libraries we need
import pandas as pd
import pyreadr

In [6]:
# Import the data from the source CSV file the 'Create a Pandas DataFrame From a CSV File' recipe and take a peek
credit_card_default_raw = pd.read_csv('./Data/credit_card_default.csv')
result = pyreadr.read_r('./Data/credit_card_default_eng.RData') 
credit_card_default_eng = result[None]
credit_card_default = credit_card_default_eng.copy()
df_flags = credit_card_default_raw['data.group']
credit_card_default = credit_card_default.join(df_flags)

credit_card_default.head()

,DEFAULT,age_bins,bill_avg,payment_avg,pay_ratio1,pay_ratio2,pay_ratio3,pay_ratio4,pay_ratio5,ratio_avg,...,util3,util4,util5,util6,util_avg,balance_growth_6mo,bill_max,payment_max,pay_max,data.group
0,1,21-30,1284.000000,114.833333,0.000000,1.000000,1.000000,1.000000,1.000000,0.800000,...,0.034450,0.000000,0.000000,0.000000,0.064200,-0.195650,3913,689,2.0,1
1,1,21-30,2846.166667,833.333333,0.000000,0.372856,0.305623,0.289436,0.000000,0.193583,...,0.022350,0.027267,0.028792,0.028792,0.023987,0.006442,3455,2000,2.0,3
2,0,31-40,16942.166667,1836.333333,0.108220,0.110628,0.069779,0.066899,0.064313,0.083968,...,0.150656,0.159233,0.166089,0.166089,0.187133,-0.158789,29239,5000,0.0,1
3,0,31-40,38555.666667,1398.000000,0.041465,0.040961,0.042382,0.037985,0.036180,0.039794,...,0.985820,0.566280,0.579180,0.579180,0.769153,-0.360620,49291,2019,0.0,3
4,0,51-60,18223.166667,9841.500000,0.352734,1.023608,0.477555,0.470072,0.036015,0.471997,...,0.716700,0.418800,0.382920,0.382920,0.364513,0.210580,35835,36681,0.0,3


## Available Columns

In [7]:
# Create a DataFrame of the columns in the credit_card_default dataframe
columns = pd.DataFrame(list(credit_card_default.columns.values))
columns

,0
0,DEFAULT
1,age_bins
2,bill_avg
3,payment_avg
4,pay_ratio1
5,pay_ratio2
6,pay_ratio3
7,pay_ratio4
8,pay_ratio5
9,ratio_avg


## Data Types

In [8]:
# Create a DataFrame of the data type of each column
data_types = pd.DataFrame(credit_card_default.dtypes,
                          columns=['Data Type'])
data_types

,Data Type
DEFAULT,int32
age_bins,category
bill_avg,float64
payment_avg,float64
pay_ratio1,float64
pay_ratio2,float64
pay_ratio3,float64
pay_ratio4,float64
pay_ratio5,float64
ratio_avg,float64


## Count of Missing Values in Each Column

In [9]:
# Create a DataFrame with the count of missing values in each column
missing_data_counts = pd.DataFrame(credit_card_default.isnull().sum(),
                                   columns=['Missing Values'])
missing_data_counts

,Missing Values
DEFAULT,0
age_bins,0
bill_avg,0
payment_avg,0
pay_ratio1,0
pay_ratio2,0
pay_ratio3,0
pay_ratio4,0
pay_ratio5,0
ratio_avg,0


## Count of Present Values in Each Column

In [10]:
# Create a DataFrame with the count of present values in each column
present_data_counts = pd.DataFrame(credit_card_default.count(),
                                   columns=['Present Values'])
present_data_counts

,Present Values
DEFAULT,30000
age_bins,30000
bill_avg,30000
payment_avg,30000
pay_ratio1,30000
pay_ratio2,30000
pay_ratio3,30000
pay_ratio4,30000
pay_ratio5,30000
ratio_avg,30000


## Number of Unique Values Per-Column

In [11]:
# Create DataFrame with the count of unique values in each column
unique_value_counts = pd.DataFrame(columns=['Unique Values'])
for v in list(credit_card_default.columns.values):
    unique_value_counts.loc[v] = [credit_card_default[v].nunique()]
unique_value_counts

,Unique Values
DEFAULT,2
age_bins,6
bill_avg,27370
payment_avg,19180
pay_ratio1,20209
pay_ratio2,20042
pay_ratio3,19411
pay_ratio4,18580
pay_ratio5,18025
ratio_avg,24820


## The Minimum Value In Each Column

In [17]:
credit_card_default.age_bins = credit_card_default.age_bins.values.as_ordered() # need to convert

In [18]:
# Create a DataFrame with the minimum value in each column
minimum_values = pd.DataFrame(columns=['Minimum Value'])
for v in list(credit_card_default.columns.values):
    minimum_values.loc[v] = [credit_card_default[v].min()]
minimum_values

,Minimum Value
DEFAULT,0
age_bins,21-30
bill_avg,-56043.166667
payment_avg,0.0
pay_ratio1,0.0
pay_ratio2,0.0
pay_ratio3,0.0
pay_ratio4,0.0
pay_ratio5,0.0
ratio_avg,0.0


## The Maximum Value In Each Column

In [19]:
# Create a DataFrame with the minimum value in each column
maximum_values = pd.DataFrame(columns=['Maximum Value'])
for v in list(credit_card_default.columns.values):
    maximum_values.loc[v] = [credit_card_default[v].max()]
maximum_values

,Maximum Value
DEFAULT,1
age_bins,71-80
bill_avg,877313.833333
payment_avg,627344.333333
pay_ratio1,4444.333333
pay_ratio2,5001.0
pay_ratio3,4444.333333
pay_ratio4,129.705128
pay_ratio5,690.655172
ratio_avg,2667.199955


# Bring It All Together

In [20]:
# Merge all the DataFrames together by the index
data_quality_report = data_types.join(present_data_counts).join(missing_data_counts).join(unique_value_counts).join(minimum_values).join(maximum_values)

In [21]:
# Print out a nice report
print("\nData Quality Report")
print("Total records: {}".format(len(credit_card_default.index)))
data_quality_report


Data Quality Report
Total records: 30000


,Data Type,Present Values,Missing Values,Unique Values,Minimum Value,Maximum Value
DEFAULT,int32,30000,0,2,0,1
age_bins,category,30000,0,6,21-30,71-80
bill_avg,float64,30000,0,27370,-56043.166667,877313.833333
payment_avg,float64,30000,0,19180,0.0,627344.333333
pay_ratio1,float64,30000,0,20209,0.0,4444.333333
pay_ratio2,float64,30000,0,20042,0.0,5001.0
pay_ratio3,float64,30000,0,19411,0.0,4444.333333
pay_ratio4,float64,30000,0,18580,0.0,129.705128
pay_ratio5,float64,30000,0,18025,0.0,690.655172
ratio_avg,float64,30000,0,24820,0.0,2667.199955
